# Globalne wyjaśnienia modelu

### Zuzanna Glinka
### Karol Degórski
### Adrian Kamiński

## Wybrany problem 

W ramach projektu zajmiemy się przede wszystkim analizą wyjaśniem związanych z kształtem.

Na poniższym ryskunku prezentujemy postawiony przez nas problem. Naszym celem jest wykrywanie czy duża figura (kwadrat, trójkąt lub okrąg) jest złożona z tylko i wyłącznie takich samych małych figur. Będzie to zatem problem klasyfikacji na 4 klasy:
- 0: koło
- 1: błędny
- 2: kwadrat
- 3: trójkąt

Przykłady wykorzystanych przez nas obrazów

![](https://raw.githubusercontent.com/adrian22311/2022L-WB-XIC/main/projects/Glinka_Degórski_Kamiński/images_example.png)

Wytrenowaliśmy kilka modeli (resnet18, resnet34, vgg11, efficientnet_b0), accuracy na poziomie 0.88-0.92

Wyjaśniamy model Resnet18. 

Ponieważ generowaliśmy dane samodzielnie mieliśmy dostęp do pełnej informacji jak każdy obraz powstał (pozycja, wielkość, kształt, kolor każdej z figur). Dlatego też udało nam się wygenerować pliki takie jak poniższy:

In [214]:
pd.read_csv('../data/details/test/circle/000000_circleMulti1.csv')

,x,y,shape,size,color,label
0,0.823762,0.555742,circle,0.066667,yellow,true circle
1,0.813852,0.631955,circle,0.050000,red,true circle
2,0.725576,0.704774,circle,0.066667,red,true circle
3,0.600747,0.763981,circle,0.075000,yellow,true circle
4,0.509717,0.778547,circle,0.050000,red,true circle
5,0.430880,0.757554,circle,0.083333,red,true circle
6,0.368483,0.677482,circle,0.050000,blue,true circle
7,0.351999,0.556972,circle,0.041667,red,true circle
8,0.353852,0.417086,circle,0.083333,red,true circle
9,0.378473,0.361297,circle,0.050000,red,true circle


Następnie wybraliśmy po 100 obserwacji z każdej klasy które model dobrze sklasyfikował. I dla tych obrazów, wyznaczyliśmy, używająć shapa, wyjaśnienia każdego z pikseli i korzystając z powyższej tabeli dopasowaliśmy do każdej z figur, jaki miała ona wpływ.

Na tym etapie mieliśmy już csvki jak poniższa:

In [217]:
pd.read_csv('./csv/_0_0_importance.csv').head()

,x,y,shape,size,color,label,class,isTrue,mainShape,size_d,Importance,img_idx
0,0.725741,0.578848,circle,0.041667,yellow,true circle,0,True,circle,small,0.356176,185
1,0.669418,0.672296,circle,0.075000,yellow,true circle,0,True,circle,big,0.895480,185
2,0.606276,0.782110,circle,0.041667,blue,true circle,0,True,circle,small,0.650871,185
3,0.441361,0.821049,circle,0.075000,yellow,true circle,0,True,circle,big,0.342942,185
4,0.306318,0.747182,circle,0.041667,blue,true circle,0,True,circle,small,0.608950,185


### Analiza otrzymanych danych
### Agregacja i inne operacje

In [1]:
import pickle
import numpy as np
import pandas as pd

In [117]:
CLASSES = ['Circle', 'None', 'Square', 'Triangle']

In [218]:
def aggregate(data, by, agg_measures, columns='Importance', where=None):
    x = data if where is None else data[where]

    return x.groupby(by=['img_idx'] + by).agg(agg_measures).groupby(by=by).mean()[columns]

In [120]:
def aggregate_ratio(data, by, agg_measures, columns='Importance', where=None):

    return aggregate(
        data=data,
        by=by,
        agg_measures=agg_measures,
        columns=columns,
        where=where
    ) / aggregate(
        data=data,
        by=by,
        agg_measures=agg_measures,
        columns=columns,
        where=None
    )


In [139]:
def create_merged_results(datasets, aggregate_func, wheres=None, **kwargs):
    results = []
    if wheres is None:
        wheres = [None] * len(datasets)
    for data, where in zip(datasets, wheres):
        x = aggregate_func(data=data, **kwargs, where=where)
        x['class'] = CLASSES[data['class'][0]]
        results.append(x)
        
    con = pd.concat(results)
    return con.set_index(['class', con.index]).sort_index()

##### Wczytywanie

In [220]:
_0 = pd.read_csv('./csv/_0_0_importance.csv')
_1 = pd.read_csv('./csv/_1_1_importane.csv')
_2 = pd.read_csv('./csv/_2_2_importane.csv')
_3 = pd.read_csv('./csv/_3_3_importane.csv')

datasets = [_0,_1,_2,_3]

### Średni wpływ ze względu na kształt

In [144]:
create_merged_results(datasets=[_0,_1,_2,_3], aggregate_func=aggregate, by=['shape'], agg_measures=['min','max', 'mean'])

min       max      mean
class    shape                                 
Circle   circle   -0.553896  0.830180  0.169140
         square   -0.411557  0.471737  0.007745
         triangle -0.221118  0.188074 -0.018540
None     circle   -0.629873  0.488206 -0.096776
         square   -0.508448  0.737067  0.114000
         triangle -0.198143  0.321154  0.068725
Square   circle   -0.365289  0.441253  0.018298
         square   -0.599299  0.881642  0.155575
         triangle -0.141003  0.186853  0.027493
Triangle circle   -0.401160  0.493134  0.028857
         square   -0.477613  0.487594  0.009031
         triangle -0.304981  0.447007  0.074445

### Średni wpływ ze względu na wielkość i kształt

In [128]:
create_merged_results(datasets=[_0,_1,_2,_3], aggregate_func=aggregate, by=['size_d'], agg_measures=['min','max', 'mean'])

min       max      mean
class    size_d                              
Circle   big    -0.391055  0.755660  0.194732
         medium -0.446473  0.659777  0.117306
         small  -0.423116  0.685096  0.123463
None     big    -0.575319  0.636644  0.023284
         medium -0.560240  0.554009  0.007042
         small  -0.530961  0.569892  0.022424
Square   big    -0.455840  0.765246  0.155293
         medium -0.455885  0.689616  0.131739
         small  -0.425272  0.703494  0.133727
Triangle big    -0.405528  0.536682  0.063085
         medium -0.323496  0.513809  0.077950
         small  -0.345008  0.532883  0.082336

In [151]:
create_merged_results(datasets=datasets, aggregate_func=aggregate, by=['shape', 'size_d'], agg_measures=['min','max', 'mean'])

min       max      mean
class    shape    size_d                              
Circle   circle   big    -0.273843  0.692455  0.231685
                  medium -0.339140  0.614302  0.144424
                  small  -0.354063  0.620969  0.145262
         square   big    -0.257279  0.324376  0.030317
                  medium -0.265646  0.123521 -0.073533
                  small  -0.112815  0.250156  0.062198
         triangle big    -0.113889  0.107939 -0.003676
                  medium -0.071156  0.132011  0.027457
                  small  -0.155361  0.094320 -0.023595
None     circle   big    -0.405402  0.213369 -0.098006
                  medium -0.408907  0.194212 -0.109971
                  small  -0.352390  0.272328 -0.034524
         square   big    -0.251434  0.496744  0.120486
                  medium -0.333741  0.433207  0.051369
                  small  -0.311167  0.422703  0.046042
         triangle big    -0.111302  0.201930  0.045222
                  medium -0.107858  0.201404  0.054833
                  small  -0.095027  0.172425  0.040008
Square   circle   big    -0.130703  0.244144  0.062384
                  medium -0.165611  0.184345  0.001685
                  small  -0.106286  0.256812  0.075104
         square   big    -0.376719  0.728548  0.170285
                  medium -0.381463  0.649373  0.150688
                  small  -0.375432  0.663615  0.143433
         triangle big    -0.030466  0.155439  0.062183
                  medium -0.069178  0.113049  0.025044
                  small  -0.102496  0.141651  0.022080
Triangle circle   big    -0.285667  0.228529 -0.020392
                  medium -0.132330  0.273349  0.067321
                  small  -0.197695  0.301721  0.036006
         square   big    -0.299967  0.321365  0.016742
                  medium -0.234074  0.281775  0.015164
                  small  -0.216520  0.253105  0.018410
         triangle big    -0.188154  0.318550  0.066322
                  medium -0.182330  0.316276  0.072343
                  small  -0.201128  0.370201  0.078612

### Ile procent konkretnych kształtów ma dodatni wpływ na predykcję

In [171]:
create_merged_results(datasets=datasets, aggregate_func=aggregate_ratio,
                      by=['size_d'], agg_measures=['count'],
                      wheres=[data.Importance > 0 for data in datasets])

count
class    size_d          
Circle   big     0.593646
         medium  0.534188
         small   0.564732
None     big     0.519052
         medium  0.522624
         small   0.541614
Square   big     0.591682
         medium  0.575576
         small   0.569511
Triangle big     0.588235
         medium  0.590613
         small   0.576823

### Jak to wygląda jak podzielimy to jeszcze ze względu na kształt

In [172]:
create_merged_results(datasets=datasets, aggregate_func=aggregate_ratio,
                      by=['size_d', 'shape'], agg_measures=['count'],
                      wheres=[data.Importance > 0 for data in datasets])

count
class    size_d shape             
Circle   big    circle    0.643559
                square    0.669261
                triangle  0.662722
         medium circle    0.586562
                square    0.708910
                triangle  0.736897
         small  circle    0.612014
                square    0.726496
                triangle  0.757551
None     big    circle    0.580645
                square    0.641359
                triangle  0.729592
         medium circle    0.560832
                square    0.613361
                triangle  0.734273
         small  circle    0.641860
                square    0.586081
                triangle  0.752275
Square   big    circle    0.654762
                square    0.617862
                triangle  0.878716
         medium circle    0.698219
                square    0.600060
                triangle  0.890961
         small  circle    0.721798
                square    0.585586
                triangle  0.777171
Triangle big    circle    0.637009
                square    0.723684
                triangle  0.676625
         medium circle    0.742529
                square    0.660750
                triangle  0.678286
         small  circle    0.670671
                square    0.730968
                triangle  0.632331

### Jaki jest stostunek figur na obrazku które mają pozywyny wpływ, jaki on jest?

In [198]:
create_merged_results(datasets=datasets, aggregate_func=aggregate_ratio,
                      by=['isTrue'], agg_measures=['count'],
                      wheres=[data.Importance > 0 for data in datasets])

count
class    isTrue          
Circle   False   0.505162
         True    0.615385
None     False   0.517002
Square   False   0.532975
         True    0.612472
Triangle False   0.538048
         True    0.600931

In [199]:
create_merged_results(datasets=datasets, aggregate_func=aggregate,
                      by=['isTrue'], agg_measures=['min', 'max', 'mean'],
                      wheres=[data.Importance > 0 for data in datasets])

min       max      mean
class    isTrue                              
Circle   False   0.054737  0.741305  0.346495
         True    0.093478  0.771764  0.416223
None     False   0.041684  0.799374  0.344201
Square   False   0.057037  0.683448  0.320224
         True    0.070007  0.869354  0.414985
Triangle False   0.097046  0.731270  0.375173
         True    0.083282  0.492244  0.273749

### Jaki jest stostunek figur na obrazku które mają negatywny wpływ, jaki on jest?

In [201]:
create_merged_results(datasets=datasets, aggregate_func=aggregate_ratio,
                      by=['isTrue'], agg_measures=['count'],
                      wheres=[data.Importance < 0 for data in datasets])

count
class    isTrue          
Circle   False   0.494838
         True    0.427350
None     False   0.482998
Square   False   0.473422
         True    0.412264
Triangle False   0.461952
         True    0.580965

In [203]:
create_merged_results(datasets=datasets, aggregate_func=aggregate,
                      by=['isTrue'], agg_measures=['min', 'max', 'mean'],
                      wheres=[data.Importance < 0 for data in datasets])

min       max      mean
class    isTrue                              
Circle   False  -0.644269 -0.065046 -0.309190
         True   -0.572265 -0.076269 -0.286880
None     False  -0.766412 -0.064808 -0.344842
Square   False  -0.599331 -0.048182 -0.276598
         True   -0.586475 -0.078054 -0.278764
Triangle False  -0.667575 -0.085036 -0.331044
         True   -0.424785 -0.059857 -0.220793

### Średni wpływ koloru konkretnych figur na predycję

In [206]:
create_merged_results(datasets=datasets, aggregate_func=aggregate,
                      by=['color'], agg_measures=['mean'])

mean
class    color           
Circle   blue    0.173565
         red     0.149544
         yellow  0.118836
None     blue    0.035612
         red    -0.004836
         yellow  0.021891
Square   blue    0.100305
         red     0.148393
         yellow  0.173712
Triangle blue    0.075439
         red     0.069542
         yellow  0.078144

### Średni wpływ koloru i kształtu konkretnych figur na predycję

In [207]:
create_merged_results(datasets=datasets, aggregate_func=aggregate,
                      by=['color', 'shape'], agg_measures=['mean'])

mean
class    color  shape             
Circle   blue   circle    0.198428
                square    0.037948
                triangle  0.012097
         red    circle    0.181888
                square   -0.019762
                triangle -0.017217
         yellow circle    0.132266
                square    0.009566
                triangle -0.070852
None     blue   circle   -0.062155
                square    0.133927
                triangle  0.040986
         red    circle   -0.096760
                square    0.054114
                triangle  0.060724
         yellow circle   -0.086134
                square    0.086554
                triangle  0.042736
Square   blue   circle    0.055685
                square    0.106275
                triangle  0.011350
         red    circle   -0.015962
                square    0.172266
                triangle  0.058764
         yellow circle    0.064703
                square    0.168526
                triangle  0.048176
Triangle blue   circle    0.017371
                square    0.123262
                triangle  0.065916
         red    circle    0.014046
                square   -0.057292
                triangle  0.076284
         yellow circle    0.027357
                square    0.016101
                triangle  0.078115